In [1]:
import sympy as sym
import sympy.utilities.codegen as cgen
import sympy.codegen.rewriting as copt
from IPython.display import display
from IPython.display import Code

In [2]:
def gradient(f, v):
    return sym.Matrix([f]).jacobian(v)

def hessian(f, v):
    return sym.hessian(f, v)

def generate(export):
    routines = [(name, copt.optimize(sym.simplify(expr), copt.optims_c99)) for name, expr in export]
    generator = cgen.CCodeGen(cse=True)
    [(_, code), (_, _)] = cgen.codegen(routines, header=False, empty=True, code_gen=generator)
    # TODO: Further post-process code for cleanup (e.g. convert to Eigen types).
    display(Code(code))
    return code

## ST-VK Hyperelastic Model

In [3]:
# Define input variables and constants.

f00, f01, f02, f10, f11, f12, f20, f21, f22 = f = sym.symbols('f00 f01 f02 f10 f11 f12 f20 f21 f22')
lame1, lame2 = sym.symbols('lambda mu', constants = True)

# Define expression to evaluate.

F = sym.Matrix([[ f00, f01, f02 ], [ f10, f11, f12 ], [ f20, f21, f22 ]])
I = sym.eye(3)
E1 = (F.T * F - I) / 2
E2 = E1 * E1

trE1 = sym.Trace(E1).doit() # Explicitly evaluate trace. Sympy has problems differentiating
trE2 = sym.Trace(E2).doit() # it otherwise.

U = lame1 * (trE1 ** 2) / 2 + lame2 * trE2
dUdF = gradient(U, f)
d2UdF2 = hessian(U, f)

# Define specific routines to export to code.

export = [("U", U), ("dUdF", dUdF), ("d2UdF2", d2UdF2)]

# Display.

print('Energy')
display(U)
print('Gradient')
display(dUdF)
print('Hessian')
display(d2UdF2)
print('Code')
generate(export);


Energy


lambda*(f00**2/2 + f01**2/2 + f02**2/2 + f10**2/2 + f11**2/2 + f12**2/2 + f20**2/2 + f21**2/2 + f22**2/2 - 3/2)**2/2 + mu*(2*(f00*f01/2 + f10*f11/2 + f20*f21/2)**2 + 2*(f00*f02/2 + f10*f12/2 + f20*f22/2)**2 + 2*(f01*f02/2 + f11*f12/2 + f21*f22/2)**2 + (f00**2/2 + f10**2/2 + f20**2/2 - 1/2)**2 + (f01**2/2 + f11**2/2 + f21**2/2 - 1/2)**2 + (f02**2/2 + f12**2/2 + f22**2/2 - 1/2)**2)

Gradient


Matrix([[f00*lambda*(f00**2/2 + f01**2/2 + f02**2/2 + f10**2/2 + f11**2/2 + f12**2/2 + f20**2/2 + f21**2/2 + f22**2/2 - 3/2) + mu*(2*f00*(f00**2/2 + f10**2/2 + f20**2/2 - 1/2) + 2*f01*(f00*f01/2 + f10*f11/2 + f20*f21/2) + 2*f02*(f00*f02/2 + f10*f12/2 + f20*f22/2)), f01*lambda*(f00**2/2 + f01**2/2 + f02**2/2 + f10**2/2 + f11**2/2 + f12**2/2 + f20**2/2 + f21**2/2 + f22**2/2 - 3/2) + mu*(2*f00*(f00*f01/2 + f10*f11/2 + f20*f21/2) + 2*f01*(f01**2/2 + f11**2/2 + f21**2/2 - 1/2) + 2*f02*(f01*f02/2 + f11*f12/2 + f21*f22/2)), f02*lambda*(f00**2/2 + f01**2/2 + f02**2/2 + f10**2/2 + f11**2/2 + f12**2/2 + f20**2/2 + f21**2/2 + f22**2/2 - 3/2) + mu*(2*f00*(f00*f02/2 + f10*f12/2 + f20*f22/2) + 2*f01*(f01*f02/2 + f11*f12/2 + f21*f22/2) + 2*f02*(f02**2/2 + f12**2/2 + f22**2/2 - 1/2)), f10*lambda*(f00**2/2 + f01**2/2 + f02**2/2 + f10**2/2 + f11**2/2 + f12**2/2 + f20**2/2 + f21**2/2 + f22**2/2 - 3/2) + mu*(2*f10*(f00**2/2 + f10**2/2 + f20**2/2 - 1/2) + 2*f11*(f00*f01/2 + f10*f11/2 + f20*f21/2) + 2*f12*(

Hessian


Matrix([
[f00**2*lambda + lambda*(f00**2/2 + f01**2/2 + f02**2/2 + f10**2/2 + f11**2/2 + f12**2/2 + f20**2/2 + f21**2/2 + f22**2/2 - 3/2) + mu*(3*f00**2 + f01**2 + f02**2 + f10**2 + f20**2 - 1),                                                                                                                                     f00*f01*lambda + mu*(2*f00*f01 + f10*f11 + f20*f21),                                                                                                                                     f00*f02*lambda + mu*(2*f00*f02 + f10*f12 + f20*f22),                                                                                                                                     f00*f10*lambda + mu*(2*f00*f10 + f01*f11 + f02*f12),                                                                                                                                                             f00*f11*lambda + f01*f10*mu,                                                                  

Code


#include "U.h"
#include <math.h>

double U(double f00, double f01, double f02, double f10, double f11, double f12, double f20, double f21, double f22, double lambda, double mu) {
   const double x0 = pow(f00, 2) + pow(f10, 2) + pow(f20, 2);
   const double x1 = pow(f01, 2) + pow(f11, 2) + pow(f21, 2);
   const double x2 = pow(f02, 2) + pow(f12, 2) + pow(f22, 2);

   double U_result;
   U_result = (1.0/8.0)*lambda*pow(x0 + x1 + x2 - 3, 2) + (1.0/4.0)*mu*(pow(x0 - 1, 2) + pow(x1 - 1, 2) + pow(x2 - 1, 2) + 2*pow(f00*f01 + f10*f11 + f20*f21, 2) + 2*pow(f00*f02 + f10*f12 + f20*f22, 2) + 2*pow(f01*f02 + f11*f12 + f21*f22, 2));
   return U_result;

}

void dUdF(double f00, double f01, double f02, double f10, double f11, double f12, double f20, double f21, double f22, double lambda, double mu, double *out_6633082485279340616) {
   const double x0 = pow(f00, 2) + pow(f10, 2) + pow(f20, 2);
   const double x1 = pow(f01, 2) + pow(f11, 2) + pow(f21, 2);
   const double x2 = pow(f02, 2) + pow(f12, 2) + pow(f22, 2);
   const double x3 = (1.0/2.0)*lambda*(x0 + x1 + x2 - 3);
   const double x4 = f00*f01 + f10*f11 + f20*f21;
   const double x5 = f00*f02 + f10*f12 + f20*f22;
   const double x6 = x0 - 1;
   const double x7 = f01*f02 + f11*f12 + f21*f22;
   const double x8 = x1 - 1;
   const double x9 = x2 - 1;

   out_6633082485279340616[0] = f00*x3 + mu*(f00*x6 + f01*x4 + f02*x5);
   out_6633082485279340616[1] = f01*x3 + mu*(f00*x4 + f01*x8 + f02*x7);
   out_6633082485279340616[2] = f02*x3 + mu*(f00*x5 + f01*x7 + f02*x9);
   out_6633082485279340616[3] = f10*x3 + mu*(f10*x6 + f11*x4 + f12*x5);
   out_6633082485279340616[4] = f11*x3 + mu*(f10*x4 + f11*x8 + f12*x7);
   out_6633082485279340616[5] = f12*x3 + mu*(f10*x5 + f11*x7 + f12*x9);
   out_6633082485279340616[6] = f20*x3 + mu*(f20*x6 + f21*x4 + f22*x5);
   out_6633082485279340616[7] = f21*x3 + mu*(f20*x4 + f21*x8 + f22*x7);
   out_6633082485279340616[8] = f22*x3 + mu*(f20*x5 + f21*x7 + f22*x9);

}

void d2UdF2(double f00, double f01, double f02, double f10, double f11, double f12, double f20, double f21, double f22, double lambda, double mu, double *out_1333097080565706461) {
   const double x0 = pow(f00, 2);
   const double x1 = pow(f01, 2);
   const double x2 = pow(f10, 2);
   const double x3 = x1 + x2;
   const double x4 = pow(f20, 2);
   const double x5 = pow(f02, 2);
   const double x6 = x5 - 1;
   const double x7 = x4 + x6;
   const double x8 = pow(f12, 2);
   const double x9 = pow(f22, 2);
   const double x10 = pow(f21, 2);
   const double x11 = pow(f11, 2);
   const double x12 = x0 + x11;
   const double x13 = x10 + x12;
   const double x14 = (1.0/2.0)*lambda*(x13 + x3 + x4 + x5 + x8 + x9 - 3);
   const double x15 = f00*f01;
   const double x16 = f10*f11;
   const double x17 = f20*f21;
   const double x18 = lambda*x15 + mu*(2*x15 + x16 + x17);
   const double x19 = f00*f02;
   const double x20 = f10*f12;
   const double x21 = f20*f22;
   const double x22 = lambda*x19 + mu*(2*x19 + x20 + x21);
   const double x23 = f00*f10;
   const double x24 = f01*f11;
   const double x25 = f02*f12;
   const double x26 = lambda*x23 + mu*(2*x23 + x24 + x25);
   const double x27 = f00*lambda;
   const double x28 = f10*mu;
   const double x29 = f01*x28 + f11*x27;
   const double x30 = f02*x28 + f12*x27;
   const double x31 = f00*f20;
   const double x32 = f01*f21;
   const double x33 = f02*f22;
   const double x34 = lambda*x31 + mu*(2*x31 + x32 + x33);
   const double x35 = f20*mu;
   const double x36 = f01*x35 + f21*x27;
   const double x37 = f02*x35 + f22*x27;
   const double x38 = f01*f02;
   const double x39 = f11*f12;
   const double x40 = f21*f22;
   const double x41 = lambda*x38 + mu*(2*x38 + x39 + x40);
   const double x42 = f11*mu;
   const double x43 = f01*lambda;
   const double x44 = f00*x42 + f10*x43;
   const double x45 = lambda*x24 + mu*(x23 + 2*x24 + x25);
   const double x46 = f02*x42 + f12*x43;
   const double x47 = f21*mu;
   const double x48 = f00*x47 + f20*x43;
   const double x49 = 

## ST-VK Dissipation Potential

In [4]:
# Define input variables and constants.

f00, f01, f02, f10, f11, f12, f20, f21, f22 = f = sym.symbols('f00 f01 f02 f10 f11 f12 f20 f21 f22')
g00, g01, g02, g10, g11, g12, g20, g21, g22 = g = sym.symbols('g00 g01 g02 g10 g11 g12 g20 g21 g22')
lame1, lame2 = sym.symbols('lambda mu', constants = True)

# Define expression to evaluate.

F = sym.Matrix([[ f00, f01, f02 ], [ f10, f11, f12 ], [ f20, f21, f22 ]])
G = sym.Matrix([[ g00, g01, g02 ], [ g10, g11, g12 ], [ g20, g21, g22 ]])
I = sym.eye(3)
E1 = (G.T * F + F.T * G) / 2
E2 = E1 * E1

trE1 = sym.Trace(E1).doit()
trE2 = sym.Trace(E2).doit()

R = lame1 * (trE1 ** 2) / 2 + lame2 * trE2
dRdFG = gradient(R, [ *f, *g ])
d2RdFG2 = hessian(R, [ *f, *g ])

# Define specific routines to export to code.

export = [("R", R), ("dRdFG", dRdFG), ("d2RdF2", d2RdFG2)]

# Display.

print('Dissipation potential')
display(R)
print('Gradient')
display(dRdFG)
print('Hessian')
display(d2RdFG2)
print('Code')
generate(export);


Dissipation potential


lambda*(f00*g00 + f01*g01 + f02*g02 + f10*g10 + f11*g11 + f12*g12 + f20*g20 + f21*g21 + f22*g22)**2/2 + mu*((f00*g00 + f10*g10 + f20*g20)**2 + (f01*g01 + f11*g11 + f21*g21)**2 + (f02*g02 + f12*g12 + f22*g22)**2 + 2*(f00*g01/2 + f01*g00/2 + f10*g11/2 + f11*g10/2 + f20*g21/2 + f21*g20/2)**2 + 2*(f00*g02/2 + f02*g00/2 + f10*g12/2 + f12*g10/2 + f20*g22/2 + f22*g20/2)**2 + 2*(f01*g02/2 + f02*g01/2 + f11*g12/2 + f12*g11/2 + f21*g22/2 + f22*g21/2)**2)

Gradient


Matrix([[g00*lambda*(f00*g00 + f01*g01 + f02*g02 + f10*g10 + f11*g11 + f12*g12 + f20*g20 + f21*g21 + f22*g22) + mu*(2*g00*(f00*g00 + f10*g10 + f20*g20) + 2*g01*(f00*g01/2 + f01*g00/2 + f10*g11/2 + f11*g10/2 + f20*g21/2 + f21*g20/2) + 2*g02*(f00*g02/2 + f02*g00/2 + f10*g12/2 + f12*g10/2 + f20*g22/2 + f22*g20/2)), g01*lambda*(f00*g00 + f01*g01 + f02*g02 + f10*g10 + f11*g11 + f12*g12 + f20*g20 + f21*g21 + f22*g22) + mu*(2*g00*(f00*g01/2 + f01*g00/2 + f10*g11/2 + f11*g10/2 + f20*g21/2 + f21*g20/2) + 2*g01*(f01*g01 + f11*g11 + f21*g21) + 2*g02*(f01*g02/2 + f02*g01/2 + f11*g12/2 + f12*g11/2 + f21*g22/2 + f22*g21/2)), g02*lambda*(f00*g00 + f01*g01 + f02*g02 + f10*g10 + f11*g11 + f12*g12 + f20*g20 + f21*g21 + f22*g22) + mu*(2*g00*(f00*g02/2 + f02*g00/2 + f10*g12/2 + f12*g10/2 + f20*g22/2 + f22*g20/2) + 2*g01*(f01*g02/2 + f02*g01/2 + f11*g12/2 + f12*g11/2 + f21*g22/2 + f22*g21/2) + 2*g02*(f02*g02 + f12*g12 + f22*g22)), g10*lambda*(f00*g00 + f01*g01 + f02*g02 + f10*g10 + f11*g11 + f12*g12 + f20*

Hessian


Matrix([
[                                                                                                                               g00**2*lambda + mu*(2*g00**2 + g01**2 + g02**2),                                                                                                                                                    g00*g01*lambda + g00*g01*mu,                                                                                                                                                    g00*g02*lambda + g00*g02*mu,                                                                                                                            g00*g10*lambda + mu*(2*g00*g10 + g01*g11 + g02*g12),                                                                                                                                                    g00*g11*lambda + g01*g10*mu,                                                                                                               

Code


#include "R.h"
#include <math.h>

double R(double f00, double f01, double f02, double f10, double f11, double f12, double f20, double f21, double f22, double g00, double g01, double g02, double g10, double g11, double g12, double g20, double g21, double g22, double lambda, double mu) {
   const double x0 = f00*g00 + f10*g10 + f20*g20;
   const double x1 = f01*g01 + f11*g11 + f21*g21;
   const double x2 = f02*g02 + f12*g12 + f22*g22;

   double R_result;
   R_result = (1.0/2.0)*lambda*pow(x0 + x1 + x2, 2) + (1.0/2.0)*mu*(2*pow(x0, 2) + 2*pow(x1, 2) + 2*pow(x2, 2) + pow(f00*g01 + f01*g00 + f10*g11 + f11*g10 + f20*g21 + f21*g20, 2) + pow(f00*g02 + f02*g00 + f10*g12 + f12*g10 + f20*g22 + f22*g20, 2) + pow(f01*g02 + f02*g01 + f11*g12 + f12*g11 + f21*g22 + f22*g21, 2));
   return R_result;

}

void dRdFG(double f00, double f01, double f02, double f10, double f11, double f12, double f20, double f21, double f22, double g00, double g01, double g02, double g10, double g11, double g12, double g20, double g21, double g22, double lambda, double mu, double *out_1734148486286582961) {
   const double x0 = f00*g00 + f10*g10 + f20*g20;
   const double x1 = f01*g01 + f11*g11 + f21*g21;
   const double x2 = f02*g02 + f12*g12 + f22*g22;
   const double x3 = lambda*(x0 + x1 + x2);
   const double x4 = 2*x0;
   const double x5 = f00*g01 + f01*g00 + f10*g11 + f11*g10 + f20*g21 + f21*g20;
   const double x6 = f00*g02 + f02*g00 + f10*g12 + f12*g10 + f20*g22 + f22*g20;
   const double x7 = 2*x1;
   const double x8 = f01*g02 + f02*g01 + f11*g12 + f12*g11 + f21*g22 + f22*g21;
   const double x9 = 2*x2;

   out_1734148486286582961[0] = g00*x3 + mu*(g00*x4 + g01*x5 + g02*x6);
   out_1734148486286582961[1] = g01*x3 + mu*(g00*x5 + g01*x7 + g02*x8);
   out_1734148486286582961[2] = g02*x3 + mu*(g00*x6 + g01*x8 + g02*x9);
   out_1734148486286582961[3] = g10*x3 + mu*(g10*x4 + g11*x5 + g12*x6);
   out_1734148486286582961[4] = g11*x3 + mu*(g10*x5 + g11*x7 + g12*x8);
   out_1734148486286582961[5] = g12*x3 + mu*(g10*x6 + g11*x8 + g12*x9);
   out_1734148486286582961[6] = g20*x3 + mu*(g20*x4 + g21*x5 + g22*x6);
   out_1734148486286582961[7] = g21*x3 + mu*(g20*x5 + g21*x7 + g22*x8);
   out_1734148486286582961[8] = g22*x3 + mu*(g20*x6 + g21*x8 + g22*x9);
   out_1734148486286582961[9] = f00*x3 + mu*(f00*x4 + f01*x5 + f02*x6);
   out_1734148486286582961[10] = f01*x3 + mu*(f00*x5 + f01*x7 + f02*x8);
   out_1734148486286582961[11] = f02*x3 + mu*(f00*x6 + f01*x8 + f02*x9);
   out_1734148486286582961[12] = f10*x3 + mu*(f10*x4 + f11*x5 + f12*x6);
   out_1734148486286582961[13] = f11*x3 + mu*(f10*x5 + f11*x7 + f12*x8);
   out_1734148486286582961[14] = f12*x3 + mu*(f10*x6 + f11*x8 + f12*x9);
   out_1734148486286582961[15] = f20*x3 + mu*(f20*x4 + f21*x5 + f22*x6);
   out_1734148486286582961[16] = f21*x3 + mu*(f20*x5 + f21*x7 + f22*x8);
   out_1734148486286582961[17] = f22*x3 + mu*(f20*x6 + f21*x8 + f22*x9);

}

void d2RdF2(double f00, double f01, double f02, double f10, double f11, double f12, double f20, double f21, double f22, double g00, double g01, double g02, double g10, double g11, double g12, double g20, double g21, double g22, double lambda, double mu, double *out_1049283129846736876) {
   const double x0 = pow(g00, 2);
   const double x1 = pow(g01, 2);
   const double x2 = pow(g02, 2);
   const double x3 = lambda + mu;
   const double x4 = g00*x3;
   const double x5 = g01*x4;
   const double x6 = g02*x4;
   const double x7 = g00*g10;
   const double x8 = g01*g11;
   const double x9 = g02*g12;
   const double x10 = lambda*x7 + mu*(2*x7 + x8 + x9);
   const double x11 = g00*lambda;
   const double x12 = g10*mu;
   const double x13 = g01*x12 + g11*x11;
   const double x14 = g02*x12 + g12*x11;
   const double x15 = g00*g20;
   const double x16 = g01*g21;
   const double x17 = g02*g22;
   const double x18 = lambda*x15 + mu*(2*x15 + x16 + x17);
   const double x19 = g20*mu;
   const double x20 = g01*x19 + g21*x11;
   const double x21 = g02*x19 + g22*x11;
   const double x22 = 